In [26]:
import numpy as np

### Q1

In [125]:
from fastembed import TextEmbedding

In [126]:
query_1 = 'I just discovered the course. Can I join now?'

In [127]:
from qdrant_client import QdrantClient, models

In [128]:
model_name = 'jinaai/jina-embeddings-v2-small-en'

In [129]:
model = TextEmbedding(model_name=model_name)

In [130]:
embeddings = list(model.embed([query_1]))
query_vector = embeddings[0]

In [131]:
print("Vector dimension:", len(query_vector))
print("Min value in the vector:", np.min(query_vector))

Vector dimension: 512
Min value in the vector: -0.11726373885183883


#### verifying

In [132]:
norm_value = np.linalg.norm(query_vector)  # Should be ≈1.0
print(f"Norm of q: {norm_value}")

Norm of q: 1.0


In [133]:
dot_product = query_vector.dot(query_vector)

In [134]:
print(f"{dot_product:.1f}")

1.0


### Q2

In [54]:
doc = 'Can I still join the course after the start date?'

In [55]:
embeddings_doc = list(model.embed([doc]))
doc_vector = embeddings_doc[0]

In [58]:
print("Vector dimension:", len(doc_vector))
print("Min value in the vector:", np.min(doc_vector))

Vector dimension: 512
Min value in the vector: -0.12396320482168117


In [60]:
norm_value = np.linalg.norm(doc_vector)  # Should be ≈1.0
print(f"Norm of q: {norm_value}")

Norm of q: 1.0


In [62]:
cosine_similarity = query_vector.dot(doc_vector)

In [70]:
print(f"Cosine Similarity is {cosine_similarity:.2f}")

Cosine Similarity is 0.90


### Q3

In [88]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [73]:
cos_sims= []
for doc in documents:
   doc_vector = list(model.embed([doc['text']]))[0]
   cos_sim= np.dot(doc_vector, query_vector)
   cos_sims.append(cos_sim)

In [78]:
best_idx = np.argmax(cos_sims)

In [79]:
print(f"The index with the highest similarity with the query vector is {best_idx}")

The index with the highest similarity with the query vector is 1


#### verifying with alternate method as described in question

In [82]:
texts = [doc['text'] for doc in documents]

In [84]:
V = np.vstack(list(model.embed(texts)))

In [85]:
cos_sims = V.dot(query_vector)

In [86]:
cos_sims

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

In [87]:
best_idx = int(np.argmax(cos_sims))
print(f"The index with the highest similarity is {best_idx}")

The index with the highest similarity is 1


### Q4

In [89]:
cos_sims = []
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    full_text_vector = list(model.embed(full_text))[0]
    cos_sim= np.dot(full_text_vector, query_vector)
    cos_sims.append(cos_sim)

In [91]:
best_idx = np.argmax(cos_sims)
print(f"The index with the highest similarity with the query vector is {best_idx}")

The index with the highest similarity with the query vector is 0


In [99]:
cos_sims = [np.dot(list(model.embed(doc['question'] + ' ' + doc['text']))[0], query_vector) for doc in documents]

In [100]:
best_idx = np.argmax(cos_sims)
print(f"The index with the highest similarity with the query vector is {best_idx}")

The index with the highest similarity with the query vector is 0


### Q5

In [102]:
model_dict = {}
for model in TextEmbedding.list_supported_models():
  model_dict[model['model']] = model['dim']

In [105]:
min_dim = min(model_dict.values())

In [109]:
models_with_min_dim = [name for name, dim in model_dict.items() if dim == min_dim]

In [110]:
print(f"Lowest  dimension is {min_dim}, from model(s): {models_with_min_dim}")

Lowest  dimension is 384, from model(s): ['BAAI/bge-small-en', 'BAAI/bge-small-en-v1.5', 'snowflake/snowflake-arctic-embed-xs', 'snowflake/snowflake-arctic-embed-s', 'sentence-transformers/all-MiniLM-L6-v2', 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2']


In [120]:
naya_model_name = 'BAAI/bge-small-en'

### Q6

In [119]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


naya_documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        naya_documents.append(doc)

In [116]:
client = QdrantClient("http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[])

In [117]:
collection_name = 'llm-assignment'

In [118]:
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = 384,
        distance=models.Distance.COSINE
    )
)

True

points =[]
id=0

for doc in naya_documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['question'] + ' ' + doc['text'], model = naya_model_name),
        payload={
           "text":  doc['text'],
           "section": doc['section'],
           "question": doc['question']
        }
    )
    points.append(point)
    
    id += 1

In [124]:
client.upsert(
    collection_name = collection_name,
    points = points
)

Fetching 5 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:37<00:00,  7.42s/it]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [141]:
query_text   = "I just discovered the course. Can I join now?"
query_vector = list(TextEmbedding(model_name=naya_model_name).embed([query_text]))[0]

In [142]:
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=1
)

/tmp/ipykernel_1554/3120804225.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [154]:
results[0].payload['text']

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [147]:
top_score=results[0].score
print("Top score:", top_score)

Top score: 0.8703172


### FIN